### Unindo a prática de criar chains com os conectores 
___

## =============================================================================
## CONCEITOS FUNDAMENTAIS DO LANGCHAIN
## =============================================================================

"""
🎯 CORE COMPONENTS DE UMA APLICAÇÃO LANGCHAIN:

1. ENTRADA (Input) - Dicionário simples com os dados de entrada
2. PROMPT - Template formatado usando ChatPromptTemplate.from_messages
3. MODELO - LLM (Large Language Model) - OpenAI no nosso caso
4. PARSER DE SAÍDA - StrOutputParser() ou with_structured_output()

🔗 LCEL (LangChain Expression Language):
- Sintaxe: input | prompt | llm | output_parser
- O operador "|" (pipe) conecta os componentes em sequência
- Cada componente processa a saída do anterior
"""
---
#### Langchain Conectores


* RunnableLambda
* RunnableParallel
* RunnablePassThrough


#### Setup do ambiente

In [1]:
# --- CÉLULA 1: Setup do ambiente ---
import os
from dotenv import find_dotenv, load_dotenv

# Carregar variáveis do .env
if not load_dotenv(find_dotenv()):
    print("⚠️ Arquivo .env não encontrado. Verificando variáveis do sistema.")

# Verificar TODAS as variáveis necessárias
required_vars = ["OPENAI_API_KEY", "LANGCHAIN_API_KEY"]
missing_vars = []

for var in required_vars:
    if not os.getenv(var):
        missing_vars.append(var)

if missing_vars:
    raise ValueError(f"❌ Variáveis não encontradas: {missing_vars}")

print("✅ Variáveis de ambiente carregadas com sucesso!")
print(f"🔑 OPENAI_API_KEY: {'✓' if os.getenv('OPENAI_API_KEY') else '✗'}")
print(f"🔑 LANGCHAIN_API_KEY: {'✓' if os.getenv('LANGCHAIN_API_KEY') else '✗'}")
print(f"📊 LANGCHAIN_TRACING: {os.getenv('LANGCHAIN_TRACING_V2', 'Not set')}")
print(f"🎯 LANGCHAIN_PROJECT: {os.getenv('LANGCHAIN_PROJECT', 'Not set')}")

✅ Variáveis de ambiente carregadas com sucesso!
🔑 OPENAI_API_KEY: ✓
🔑 LANGCHAIN_API_KEY: ✓
📊 LANGCHAIN_TRACING: true
🎯 LANGCHAIN_PROJECT: Langchain studies - FabioLima


#### Configuração do LangChain e LangSmith

In [2]:
# --- CÉLULA 2: Configuração do LangChain + LangSmith ---
from langchain_openai import ChatOpenAI

# IMPORTANTE: NÃO sobrescreva as variáveis do .env aqui!
# Apenas verifique se estão definidas
print("\n🔍 Verificando configurações LangSmith:")
print(f"LANGCHAIN_TRACING_V2: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"LANGCHAIN_ENDPOINT: {os.getenv('LANGCHAIN_ENDPOINT')}")
print(f"LANGCHAIN_PROJECT: {os.getenv('LANGCHAIN_PROJECT')}")
print(f"LANGCHAIN_API_KEY: {'SET' if os.getenv('LANGCHAIN_API_KEY') else 'NOT SET'}")

# Instanciando o LLM
llm = ChatOpenAI(model="gpt-4o-mini")

print("✅ LLM configurado com sucesso!")



🔍 Verificando configurações LangSmith:
LANGCHAIN_TRACING_V2: true
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com
LANGCHAIN_PROJECT: Langchain studies - FabioLima
LANGCHAIN_API_KEY: SET
✅ LLM configurado com sucesso!


#### --- Bibliotecas Iniciais para LCEL ---

In [3]:
# --- CÉLULA 3: Imports Essenciais para LCEL ---
from langchain_core.prompts import ChatPromptTemplate # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
from langchain.schema.runnable import (
    RunnableLambda,# type: ignore
    RunnableParallel,# type: ignore
    RunnablePassthrough) # type: ignore

from langchain_core.runnables import Runnable, RunnableSerializable
from pydantic import BaseModel, Field
from typing import List, Dict, Any


**-- Teste rápido do LLM --**

In [4]:
# --- CÉLULA 4: Teste com debugging melhorado ---
def test_langsmith_connection():
    """Testa a conexão com LangSmith de forma mais detalhada"""
    print("🧪 Iniciando teste de conexão com LangSmith...")
    
    try:
        resposta = llm.invoke("Diga 'Olá mundo' em português.")
        print("✅ Resposta do modelo:", resposta.content)
        print("🎯 Se você não viu erros acima, o LangSmith está funcionando!")
        return True
    except Exception as e:
        print(f"❌ Erro durante a execução: {e}")
        return False

# Execute o teste
test_langsmith_connection()

🧪 Iniciando teste de conexão com LangSmith...
❌ Erro durante a execução: Connection error.


False

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError(\'HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Max retries exceeded with url: /info (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f2854327ed0>: Failed to resolve \\\'api.smith.langchain.com\\\' ([Errno -3] Temporary failure in name resolution)"))\'))\nContent-Length: None\nAPI Key: lsv2_********************************************40')


`Exercício: Extrair o nome de um diretor dado o nome do filme`

In [5]:
# --- CÉLULA 5: Seu exercício com melhor debugging ---
class DirectorMovie(BaseModel):
    nome: str = Field(..., description="Nome do diretor do filme")

# Modelo com temperatura para precisão
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# Entrada
input_data = {"filme": "O Poderoso Chefão"} 

# Prompt
prompt = ChatPromptTemplate([
    ("system", "Você é um especialista em filmes."),
    ("user", "Quem é o diretor do filme : {filme}? Responda apenas com o nome.")
])

# Parser estruturado
llm_structured = llm.with_structured_output(DirectorMovie)

# Chain
chain = prompt | llm_structured

print("🚀 Executando chain com LangSmith tracking...")

try:
    # Execute a chain
    result = chain.invoke(input_data)
    
    # Resultado
    print(f"🔍 Entrada: {input_data}, type: {type(input_data)}")
    print(f"🔄 Processamento: Prompt → LLM → Parser")
    print(f"✨ Resultado: {result.nome}")
    
    print("\n🎉 Sucesso! Verifique no LangSmith:")
    print(f"🌐 https://smith.langchain.com/projects/{os.getenv('LANGCHAIN_PROJECT', '').replace(' ', '%20')}")
    
except Exception as e:
    print(f"❌ Erro na execução: {e}")
    print("💡 Dica: Verifique se todas as variáveis de ambiente estão corretas")

🚀 Executando chain com LangSmith tracking...
❌ Erro na execução: Connection error.
💡 Dica: Verifique se todas as variáveis de ambiente estão corretas


`Exercício: **fazer a saída da primeira chain com letras maísculas`**

In [6]:
class DirectorMovie(BaseModel):
    nome: str = Field(..., description="Nome do diretor do filme")

# Modelo com temperatura para precisão
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# Entrada
input_data = {"filme": "O Poderoso Chefão"} 

# Prompt
prompt = ChatPromptTemplate([
    ("system", "Você é um especialista em filmes."),
    ("user", "Quem é o diretor do filme : {filme}? Responda apenas com o nome.")
])

# Parser estruturado
llm_structured = llm.with_structured_output(DirectorMovie)

# Chain
chain_movie_director = prompt | llm_structured

print("🚀 Executando chain com LangSmith tracking...")

# Transformar a saída da primeira chain com letras maísculas
to_upper = RunnableLambda(lambda input_str: input_str.upper())

# Chain composta
#* input_dicionario -> Prompt -> LLM(Pydantic) -> extrai nome do diretor do filme -> Converte para maísculas
chain_upper = chain_movie_director | to_upper
try:
    # Execute a chain
    result = chain.invoke(input_data)
    
    # Resultado
    print(f"🔍 Entrada: {input_data['filme']}, type: {type(input_data)}")
    print(f"🔄 Processamento: Prompt → LLM → Parser → to_upper")
    print(f"✨ Resultado:(String em maíuscula) {result}")
    
    print("\n🎉 Sucesso! Verifique no LangSmith:")
    print(f"🌐 https://smith.langchain.com/projects/{os.getenv('LANGCHAIN_PROJECT', '').replace(' ', '%20')}")
    
except Exception as e:
    print(f"❌ Erro na execução: {e}")
    print("💡 Dica: Verifique se todas as variáveis de ambiente estão corretas")

🚀 Executando chain com LangSmith tracking...
🔍 Entrada: O Poderoso Chefão, type: <class 'dict'>
🔄 Processamento: Prompt → LLM → Parser → to_upper
✨ Resultado:(String em maíuscula) nome='Francis Ford Coppola'

🎉 Sucesso! Verifique no LangSmith:
🌐 https://smith.langchain.com/projects/Langchain%20studies%20-%20FabioLima


Exercício: Utilizar o pydantic para extrair mais informações.

In [7]:
class MovieInfo(BaseModel):
    diretor: str = Field(..., description="Nome do diretor do filme")
    ano: int = Field(..., description="Ano de lançamento do filme")
    genero: str = Field(..., description="Gênero principal do filme")

# Modelo com temperatura para precisão
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Entrada
input_data = {"filme": "Lord of the Rings"} 

# Prompt
prompt = ChatPromptTemplate([
    ("system", 
     "Você é um especialista em cinema. "
     "Responda sempre com informações confiáveis. "
     "Se não souber, use 'Não encontrado'."),
    ("user", "Forneça informações sobre o filme {filme}.")
])

# Parser estruturado
llm_structured = llm.with_structured_output(MovieInfo)

# Chain
chain_movie_info = prompt | llm_structured

print("🚀 Executando chain com LangSmith tracking...")

# Chain composta
#* input_dicionario -> Prompt -> LLM(Pydantic) -> extrai informações definidas sobre o filme

try:
    # Execute a chain
    result = chain_movie_info.invoke(input_data)
    
    # Resultado
    print(f"🔍 Entrada: {input_data['filme']}, type: {type(input_data)}")
    print(f"🔄 Processamento: Prompt → LLM → Parser(structured output)")
    print(f"✨ Diretor:{result.diretor}")
    print(f"✨ Ano de lançamento:{result.ano}")
    print(f"✨ Gênero:{result.genero}")
    
    print("\n🎉 Sucesso! Verifique no LangSmith:")
    print(f"🌐 https://smith.langchain.com/projects/{os.getenv('LANGCHAIN_PROJECT', '').replace(' ', '%20')}")
    
except Exception as e:
    print(f"❌ Erro na execução: {e}")
    print("💡 Dica: Verifique se todas as variáveis de ambiente estão corretas")

🚀 Executando chain com LangSmith tracking...
🔍 Entrada: Lord of the Rings, type: <class 'dict'>
🔄 Processamento: Prompt → LLM → Parser(structured output)
✨ Diretor:Peter Jackson
✨ Ano de lançamento:2001
✨ Gênero:Fantasia

🎉 Sucesso! Verifique no LangSmith:
🌐 https://smith.langchain.com/projects/Langchain%20studies%20-%20FabioLima


Vamos retirar outras informações utilizando os dados anteriores.

In [8]:
class MovieInfo(BaseModel):
    diretor: str = Field(..., description="Nome do diretor do filme")
    ano: int = Field(..., description="Ano de lançamento do filme")
    genero: str = Field(..., description="Gênero principal do filme")

# Modelo com temperatura para precisão
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
llm_structured = llm.with_structured_output(MovieInfo)

# Entrada
input_data = {"filme": "The Hobbit"} 

#* Chain principal que usará a nova estrutura
#* Prompt mais poderoso para extrair a informação do filme
prompt_movie = ChatPromptTemplate([
    ("system", 
     "Você é um especialista em cinema. "
     "Responda sempre com informações confiáveis. "
     "Se não souber, use 'Não encontrado'."),
    ("user", "Forneça informações sobre o filme {filme}.")
])

chain_movie_info = ( # type: ignore
   prompt_movie | llm_structured
) 

#* Crie uma chain paralela
#* Utilizando o nome do filme como entrada
#* Criando o prompt para extrair a crítica ficticia
prompt_critica = ChatPromptTemplate([
    ("system", 
     "Você é um especialista em cinema. "
     "Responda sempre com informações confiáveis. "
     "Se não souber, use 'Não encontrado'."),
    ("user", "Escreva uma crítica de uma linha para o filme {filme}.")
])

#* A runnable parallel recebe um dicionário de runnables
chain_critica_ficticia = prompt_critica | llm | StrOutputParser() # type: ignore

parallel_chains = RunnableParallel({ # type: ignore
    "info_movie" : chain_movie_info,
    "critica_ficticia" : chain_critica_ficticia 
})

#* Executando a chain paralela
resultado_paralelo = parallel_chains.invoke(input_data) # type: ignore

info_movie = resultado_paralelo['info_movie']
critica = resultado_paralelo['critica_ficticia']

print(f"Filme: {input_data['filme']}")
print(f"Diretor: {info_movie.diretor}")
print(f"Ano: {info_movie.ano}")
print(f"Gênero: {info_movie.genero}")
print(f"Crítica Fictícia: {critica}")



Filme: The Hobbit
Diretor: Peter Jackson
Ano: 2012
Gênero: Aventura
Crítica Fictícia: "The Hobbit" oferece uma jornada visualmente deslumbrante, mas se estende demais ao tentar replicar a grandiosidade de "O Senhor dos Anéis" em três partes.


Usando a `RunnablePassthrough` para enriquecer a entrada inicial do fluxo

1. `RunnablePassthrough()`: recebe o dicionário de entrada ({"filme" : "The Hobbit"}) e o passa adiante intacto.

2. `.assign`(**movie_info = chain_movie_info**): Executa a chain_movie_info e adiciona o seu resultado (o objeto `MovieInfo`) ao dicionário, sobe nova chave - 
`"movie_info"`.

In [9]:
from langchain_core.runnables import RunnablePassthrough

#* Reutilizamos a sua chain que já busca as informações do filme
#* chain_movie_info = prompt_movie | llm_structured

#* primeira parte da chain final 
chain_enriched = RunnablePassthrough.assign(
    # A chave 'movie_info' guardará o resultado de chain_movie_info
    movie_info = chain_movie_info
)

#* Criamos um prompt simples para a sugestão 
prompt_sugestao = ChatPromptTemplate([
    ("system", """Você é um especialista em cinema. Responda de forma concisa e direta. Usando no máximo 5 palavras."""),
    ("user",   """Sugira um outro filme famoso do mesmo diretor {diretor}.""")
])

#* Criamos a chain completa, adicionando a segunda "parada"

chain_final = chain_enriched | RunnablePassthrough.assign(
    sugestao_movie = (
                    # Esta lambda extrai o nome do diretor do passo anterior
                    # e o formata para o próximo prompt.
                    (lambda x: {"diretor": x['movie_info'].diretor})
                    | prompt_sugestao
                    | llm
                    | StrOutputParser()
                )
             )

In [10]:
chain_final_sequencial = (RunnablePassthrough.assign(
    movie_info = chain_movie_info
)
| RunnablePassthrough.assign(   
    sugestao_movie = (
        (lambda x: {"diretor": x['movie_info'].diretor})
        | prompt_sugestao
        | llm
        | StrOutputParser()
      )
    )
)

In [11]:
# --- Execução e Resultado ---
resultado_final = chain_final_sequencial.invoke(input_data)

print("--- Análise Sequencial do Filme ---")
print(f"Filme Analisado: {resultado_final['filme']}")
print(f"Diretor: {resultado_final['movie_info'].diretor}")
print(f"Ano: {resultado_final['movie_info'].ano}")
print("\n--- Informação Adicional Gerada ---")
print(f"Sugestão de outro filme do mesmo diretor: {resultado_final['sugestao_movie']}")

--- Análise Sequencial do Filme ---
Filme Analisado: The Hobbit
Diretor: Peter Jackson
Ano: 2012

--- Informação Adicional Gerada ---
Sugestão de outro filme do mesmo diretor: "O Senhor dos Anéis".


## Exercícios de fixação: 
____

Objetivo: Obter informações básicas sobre um filme:

1. `input_data` = {"movie": "Um Sonho de Liberdade"}

2. `movie_info_data`= {

    "Diretor" : str,

    "Atores": list[str],

    "Ano": int
}

In [12]:
import json

class MovieInfoData(BaseModel):
    diretor: str = Field(..., description="Nome do diretor do filme")
    atores: list[str] = Field(..., description="Lista de atores do filme")
    roteiristas: list[str] = Field(..., description="Lista de roteiristas do filme")
    ano: int = Field(..., description="Ano de lançamento do filme")
    genero: str = Field(..., description="Gênero do filme")

#* Definindo a llm que será usada para extrair as informações
# Modelo com temperatura para precisão
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
llm_structured = llm.with_structured_output(MovieInfoData) # type: ignore

#* Criando o prompt para instruir o LLM a extrair as informações 
prompt_movie_data = ChatPromptTemplate([    
    ("system", """Você é um especialista em cinema. 
     Responda sempre com informações confiáveis. Se não souber, use 'Não encontrado'."""),
    ("user", """Forneça informações sobre o filme {movie}.""")
])

#* Criando a chain principal
chain_movie_data = prompt_movie_data | llm_structured # type: ignore

#* Input data
input_data = {"movie": "Um Sonho de Liberdade"}

#* Executando a chain
result = chain_movie_data.invoke(input_data) # type: ignore

#* Exibindo o resultado
print("---- Resultado da chain  ----")
print(f"Diretor: {result.diretor}")
print(f"Atores: {result.atores[:3]}")
print(f"Ano: {result.ano}")
print(f"Gênero: {result.genero}")
print(f"Roteiristas: {result.roteiristas}")

---- Resultado da chain  ----
Diretor: Frank Darabont
Atores: ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']
Ano: 1994
Gênero: Drama
Roteiristas: ['Frank Darabont']


Na tarefa anterior, extraímos algumas informações a partir do dicionário:

    * input_data = {"movie": "Um Sonho de Liberdade"}

    * output_data: result = {
        diretor: ...,
        atores: ...,
        ano: ...,
        gênero: ...,
        roteiristas: ...
    }

A nova tarefa consiste em conseguir o nome do filme na lingua inglesa, não é uma tradução é uma busca do nome do filme, portanto o input da segunda tarefa é o mesmo que da primeira, dessa forma podemos usar um `RunnableParallel`, para executar as duas tarefas ao mesmo tempo.

Para manter a estrutura system/user, a forma correta é usar o `system`para definir a persona e o comportamento e o `user`para fazer a pergunta.

In [13]:
# Prompt específico para a nova tarefa
prompt_titulo_ingles = ChatPromptTemplate([
    ("system", "Você é um pesquisador especialista em cinema que responde de forma direta. Com apenas o título do filme."),
    ("user", "Qual é o título original em inglês do filme '{movie}'?")
])

# Chain para a nova tarefa
chain_titulo_ingles = prompt_titulo_ingles | llm | StrOutputParser()

In [14]:
from langchain_core.runnables import RunnableParallel

# Sua chain original que busca os dados
# chain_movie_data = prompt_movie_data | llm_structured

# Juntando as duas tarefas independentes
tarefas_paralelas = RunnableParallel({
    "dados_filme": chain_movie_data,      # Tarefa 1
    "titulo_ingles": chain_titulo_ingles, # Tarefa 2
})

In [15]:
# O mesmo input de antes
input_data = {"movie": "Um Sonho de Liberdade"}

# Executa as duas tarefas ao mesmo tempo
resultado_final = tarefas_paralelas.invoke(input_data)

# Extrai os resultados de cada chave
dados = resultado_final['dados_filme']
titulo_en = resultado_final['titulo_ingles']

# Exibe o resultado combinado
print(f"--- Análise do Filme: {input_data['movie']} ---")
print(f"Título Original: {titulo_en}")
print(f"Diretor: {dados.diretor}")
print(f"Atores: {dados.atores}")
print(f"Ano: {dados.ano}")

--- Análise do Filme: Um Sonho de Liberdade ---
Título Original: The Shawshank Redemption.
Diretor: Frank Darabont
Atores: ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton', 'William Sadler', 'Clancy Brown', 'Gil Bellows', 'James Whitmore']
Ano: 1994


Exercício adicional ao anterior: Incluir filmografia do ator principal

1. As informações originais do filme(MovieInfoData)
2. extrair do MovieInfoData - o ator principal que deve ser o primeiro da lista do objeto de saída
3. montar prompt pra extrair filmografia


Como o fluxo acima é sequencial, podemos usar o RunnablePassthrough.assign() para orquestrar o processo


In [1]:
import os
import datetime
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv, find_dotenv

# Carregar variáveis de ambiente (necessário para a chave da API)
load_dotenv(find_dotenv())

# --- PASSO 0: DEFINIÇÃO DAS ESTRUTURAS DE DADOS (Modelos Pydantic) ---
# Estrutura para os dados iniciais do filme
class MovieInfoData(BaseModel):
    diretor: str = Field(..., description="Nome do diretor do filme")
    atores: List[str] = Field(..., description="Lista de atores principais do filme")
    ano: int = Field(..., description="Ano de lançamento do filme")

# Estrutura para a saída da filmografia
class Filmografia(BaseModel):
    filmes: List[str] = Field(description="Lista de filmes notáveis em que o ator participou")

# --- PASSO 1: CONSTRUÇÃO DAS "FERRAMENTAS" (As Sub-Chains Isoladas) ---

# Configuração do LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Ferramenta 1: Chain para buscar os dados iniciais do filme
prompt_movie_data = ChatPromptTemplate.from_template(
    """Você é um especialista em cinema. 
    Forneça o diretor, o ano e a lista dos 3 atores principais do filme {movie}."""
)
chain_busca_dados_filme = prompt_movie_data | llm.with_structured_output(MovieInfoData)

# Ferramenta 2: Chain para buscar a filmografia de um ator
ano_atual = datetime.datetime.now().year
prompt_filmografia = ChatPromptTemplate.from_template(
    """Você é um especialista em filmografia de atores.
    Liste filmes notáveis em que o ator {ator} participou, lançados estritamente depois do ano {ano_inicio} até o ano {ano_corrente}.
    Não inclua o filme do ano {ano_inicio}. Responda apenas com a lista de filmes."""
)
chain_busca_filmografia = prompt_filmografia | llm.with_structured_output(Filmografia)

# --- PASSO 2: MONTAGEM DA LINHA DE PRODUÇÃO (Pipeline Sequencial) ---

# Estação 1: Pega o nome do filme e adiciona os dados completos.
# Saída deste pipeline: {'movie': ..., 'dados_filme': MovieInfoData(...)}
pipeline_passo_1_info_filme = RunnablePassthrough.assign(
    dados_filme=chain_busca_dados_filme
)

# Estação 2: Pega o resultado anterior e adiciona o ator principal e o ano.
# Saída deste pipeline: {'movie': ..., 'dados_filme': ..., 'ator_principal': ..., 'ano_filme': ...}
pipeline_passo_2_ator_e_ano = pipeline_passo_1_info_filme | RunnablePassthrough.assign(
    ator_principal=(lambda x: x["dados_filme"].atores[0]),
    ano_filme=(lambda x: x["dados_filme"].ano)
)

# Estação 3: Pega o resultado anterior e adiciona a filmografia.
# Primeiro, criamos o "adaptador" para a Ferramenta 2.
adaptador_para_filmografia = lambda x: {
    "ator": x["ator_principal"],
    "ano_inicio": x["ano_filme"],
    "ano_corrente": ano_atual
}
# E agora, a pipeline final
pipeline_final_com_filmografia = pipeline_passo_2_ator_e_ano | RunnablePassthrough.assign(
    filmografia_resultado=(adaptador_para_filmografia | chain_busca_filmografia)
)


# --- PASSO 3: EXECUÇÃO E EXIBIÇÃO ---
input_data = {"movie": "Um Sonho de Liberdade"}
resultado_final = pipeline_final_com_filmografia.invoke(input_data)

print(f"--- Análise para o filme: {resultado_final['movie']} ---")
print(f"Ator principal identificado: {resultado_final['ator_principal']} (do filme de {resultado_final['ano_filme']})")
print("\n--- Filmografia do ator desde então ---")
# Acessamos o resultado final e o atributo .filmes do objeto Pydantic
for filme in resultado_final['filmografia_resultado'].filmes:
    print(f"- {filme}")

--- Análise para o filme: Um Sonho de Liberdade ---
Ator principal identificado: Tim Robbins (do filme de 1994)

--- Filmografia do ator desde então ---
- Dead Man Walking (1995)
- Arlington Road (1999)
- The Shawshank Redemption (1994)
- High Fidelity (2000)
- Antitrust (2001)
- The Truth About Charlie (2002)
- Mystic River (2003)
- Code 46 (2003)
- War of the Worlds (2005)
- Zathura: A Space Adventure (2005)
- Catch a Fire (2006)
- Noise (2007)
- City of Ember (2008)
- Green Lantern (2011)
- Back to 1942 (2012)
- A Perfect Day (2015)
- Marjorie Prime (2017)
- Dark Waters (2019)
- VHYes (2019)
- Here and Now (2018)


# Nossa meta: definir esta pipeline completa.
# Ela representa a sequência de TODAS as nossas operações.
pipeline_completa = (
    etapa_1_busca_paralela_de_dados
    |
    etapa_2_extracao_de_ator_e_ano
    |
    etapa_3_busca_de_filmografia
)

### Contrato de dados que serão usados no pipeline

In [2]:
from pydantic import BaseModel, Field
from typing import List

# Contrato para os dados principais do filme
class MovieInfoData(BaseModel):
    diretor: str = Field(..., description="Nome do diretor do filme")
    atores: List[str] = Field(..., description="Lista de atores principais do filme")
    ano: int = Field(..., description="Ano de lançamento do filme")

# Contrato para o resultado da filmografia
class Filmografia(BaseModel):
    filmes: List[str] = Field(description="Lista de filmes notáveis em que o ator participou")

`Passo 3: Construir Cada Etapa (Preenchendo o Esqueleto)`

Agora, vamos criar as peças para preencher nosso esqueleto do Passo 1.

    3.A - Construindo a etapa_1_busca_paralela_de_dados

        Objetivo: Buscar os dados do filme e o título em inglês ao mesmo tempo.

        Ferramenta: RunnableParallel.

    Código:

Pense no .assign() não como algo que vem depois da primeira etapa, mas como a ferramenta que constrói cada etapa sequencial que adiciona dados, incluindo a primeira.

No nosso exemplo "Top-Down":

etapa_1_busca_paralela_de_dados foi a primeira etapa, e ela não usou .assign porque era um RunnableParallel.

etapa_2_extracao_de_ator_e_ano foi a segunda etapa, e ela foi construída com um .assign.

`A regra é: sempre que você quiser pegar o dicionário que está fluindo na pipeline e adicionar uma nova chave a ele, você constrói essa etapa com um RunnablePassthrough.assign()`

Vamos formalizar o seu modelo, que é um resumo fantástico:

`chain = prompt | llm | output_parser`

Onde:

    1. A chain (A Ferramenta Completa): É a unidade de trabalho que resolve um problema específico. Ex: chain_busca_dados_filme.

    2. O prompt (O Manual de Instruções): Exatamente como você disse, é a "ferramenta para o problema a resolver". Ele define a tarefa, o contexto, as regras e o que se espera do motor.

    3. O llm (O Motor / A CPU): É o componente que processa as instruções do manual e gera o resultado bruto. É a força de trabalho intelectual.

    4. O output_parser (O Molde / O Acabamento): É a etapa final que pega o resultado bruto do motor e o formata exatamente "como queremos que o output saia", seja transformando-o em um objeto Pydantic, uma string limpa, uma lista, etc.

In [3]:
# --- CONSTRUÇÃO DA FERRAMENTA 1: chain_busca_dados_filme ---

# 1. Definição do LLM (corrigindo o nome do modelo para 'gpt-4o')
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# 2. Vinculando o LLM à nossa estrutura de dados Pydantic
llm_structured = llm.with_structured_output(MovieInfoData)

# 3. Definição do Prompt (Manual de Instruções)
# Corrigindo a sintaxe para uma lista de tuplas: [ ("role", "message") ]
prompt_movie_data = ChatPromptTemplate([
    ("system", "Você é um especialista em cinema. Responda com as informações solicitadas."),
    ("user", "Forneça o diretor, o ano e a lista dos 3 atores principais do filme {movie}.")
])

# 4. Montagem final da Ferramenta/Chain
# Usando o nome da variável que planejamos na nossa arquitetura
chain_busca_dados_filme = prompt_movie_data | llm_structured

# --- TESTE ISOLADO DA FERRAMENTA ---
# Corrigindo a sintaxe do dicionário (chaves e valores de texto precisam de aspas)
input_teste = {"movie": "Um Sonho de Liberdade"}

print("Executando teste da Ferramenta 1: chain_busca_dados_filme...")
resultado_ferramenta_1 = chain_busca_dados_filme.invoke(input_teste)

# Imprime o objeto Pydantic resultante
print(resultado_ferramenta_1)

Executando teste da Ferramenta 1: chain_busca_dados_filme...
diretor='Frank Darabont' atores=['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'] ano=1994


In [4]:
# --- CONSTRUÇÃO DA FERRAMENTA 2: chain_busca_titulo_en ---

from langchain_core.output_parsers import StrOutputParser

# 1. Definição do Prompt (Manual de Instruções)
prompt_titulo_en = ChatPromptTemplate([
    ("system", "Você é um pesquisador especialista em cinema que responde de forma concisa, direta e apenas com a informação solicitada."),
    ("user", "Qual é o título original em inglês do filme '{movie}'?")
])

# 2. Montagem final da Ferramenta/Chain
#    Usando o mesmo llm que já definimos antes.
chain_busca_titulo_en = prompt_titulo_en | llm | StrOutputParser()


# --- TESTE ISOLADO DA FERRAMENTA ---
input_teste = {"movie": "Um Sonho de Liberdade"}

print("Executando teste da Ferramenta 2: chain_busca_titulo_en...")
resultado_ferramenta_2 = chain_busca_titulo_en.invoke(input_teste)

# Imprime o resultado e o tipo para confirmar que é uma string
print(f"Resultado do teste: '{resultado_ferramenta_2}'")
print(f"Tipo do resultado: {type(resultado_ferramenta_2)}")

Executando teste da Ferramenta 2: chain_busca_titulo_en...
Resultado do teste: 'O título original em inglês do filme "Um Sonho de Liberdade" é "The Shawshank Redemption".'
Tipo do resultado: <class 'str'>


In [5]:
# --- MONTAGEM DA ETAPA 1: A Busca Paralela ---

from langchain_core.runnables import RunnableParallel

# Nossas duas ferramentas já foram definidas nos passos anteriores:
# chain_busca_dados_filme  -> Retorna um objeto MovieInfoData
# chain_busca_titulo_en    -> Retorna uma string

# Agora, criamos a etapa que executa as duas em paralelo
etapa_1_busca_paralela_de_dados = RunnableParallel({
    "dados_filme": chain_busca_dados_filme,
    "titulo_en": chain_busca_titulo_en,
})


# --- TESTE ISOLADO DA ETAPA 1 ---
input_teste = {"movie": "Um Sonho de Liberdade"}

print("Executando teste da Etapa 1: Busca Paralela...")
resultado_etapa_1 = etapa_1_busca_paralela_de_dados.invoke(input_teste)

# Exibindo o resultado completo da etapa
print("\n--- Resultado da Etapa Paralela ---")
# O resultado é um dicionário com as saídas de cada ferramenta
print(resultado_etapa_1)

print("\n--- Detalhes do Resultado ---")
print(f"Dados do Filme (tipo: {type(resultado_etapa_1['dados_filme'])}): {resultado_etapa_1['dados_filme']}")
print(f"Título em Inglês (tipo: {type(resultado_etapa_1['titulo_en'])}): '{resultado_etapa_1['titulo_en']}'")

Executando teste da Etapa 1: Busca Paralela...

--- Resultado da Etapa Paralela ---
{'dados_filme': MovieInfoData(diretor='Frank Darabont', atores=['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], ano=1994), 'titulo_en': 'O título original em inglês do filme "Um Sonho de Liberdade" é "The Shawshank Redemption".'}

--- Detalhes do Resultado ---
Dados do Filme (tipo: <class '__main__.MovieInfoData'>): diretor='Frank Darabont' atores=['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'] ano=1994
Título em Inglês (tipo: <class 'str'>): 'O título original em inglês do filme "Um Sonho de Liberdade" é "The Shawshank Redemption".'


In [ ]:
# Precisamos das duas "ferramentas" que esta etapa vai executar em paralelo
chain_busca_dados_filme = ... # (vamos definir isso em breve)
chain_busca_titulo_en = ...   # (vamos definir isso em breve)

etapa_1_busca_paralela_de_dados = RunnableParallel({
    "dados_filme": chain_busca_dados_filme,
    "titulo_en": chain_busca_titulo_en,
})

# A SAÍDA DESTA ETAPA SERÁ UM DICIONÁRIO:
# {'dados_filme': MovieInfoData(...), 'titulo_en': 'The Shawshank Redemption'}

3.B - Construindo a etapa_2_extracao_de_ator_e_ano

    * Objetivo: Extrair o ator e o ano do resultado da etapa anterior.

    * Ferramenta: RunnablePassthrough.assign com lambdas simples.

    * Código:

In [ ]:
etapa_2_extracao_de_ator_e_ano = RunnablePassthrough.assign(
    ator_principal=(lambda x: x["dados_filme"].atores[0]),
    ano_filme=(lambda x: x["dados_filme"].ano)
)

# A SAÍDA DESTA ETAPA SERÁ O DICIONÁRIO ANTERIOR, ENRIQUECIDO:
# {'dados_filme': ..., 'titulo_en': ..., 'ator_principal': 'Tim Robbins', 'ano_filme': 1994}

In [8]:
# Assumindo que a 'etapa_1_busca_paralela_de_dados' já foi criada
# Saída da Etapa 1: {'dados_filme': MovieInfoData(...), 'titulo_en': '...'}
import json

etapa_2_extracao_de_ator_e_ano = etapa_1_busca_paralela_de_dados | RunnablePassthrough.assign(
    ator_principal=(lambda x: x["dados_filme"].atores[0]),
    ano_filme=(lambda x: x["dados_filme"].ano)
)

# --- TESTE ISOLADO DA ETAPA 2 ---
input_teste = {"movie": "Um Sonho de Liberdade"}

print("Executando teste da Etapa 2: Extração de Dados...")
resultado_etapa_2 = etapa_2_extracao_de_ator_e_ano.invoke(input_teste)

print("\n--- Resultado após a Etapa 2 ---")
# O dicionário agora está mais rico
print(resultado_etapa_2)

Executando teste da Etapa 2: Extração de Dados...

--- Resultado após a Etapa 2 ---
{'dados_filme': MovieInfoData(diretor='Frank Darabont', atores=['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], ano=1994), 'titulo_en': 'O título original em inglês é "The Shawshank Redemption".', 'ator_principal': 'Tim Robbins', 'ano_filme': 1994}


3.C - Construindo a etapa_3_busca_de_filmografia

    * Objetivo: Usar os dados extraídos para buscar a filmografia.
    * Ferramenta: RunnablePassthrough.assign com uma sub-chain mais complexa.
    * Código:

In [ ]:
# Precisamos da "ferramenta" de busca de filmografia
chain_busca_filmografia = ... # (vamos definir isso em breve)

# E do "adaptador" para preparar os dados para essa ferramenta
adaptador_para_filmografia = ... # (vamos definir isso em breve)

etapa_3_busca_de_filmografia = RunnablePassthrough.assign(
    filmografia_resultado=(adaptador_para_filmografia | chain_busca_filmografia)
)

# A SAÍDA FINAL SERÁ O DICIONÁRIO COMPLETO:
# {'dados_filme': ..., 'titulo_en': ..., 'ator_principal': ..., 'ano_filme': ..., 'filmografia_resultado': Filmografia(...)}

Com prazer! Sua última frase é a conclusão perfeita de todo o nosso exercício. Você resumiu a função e a importância do RunnablePassthrough.assign de forma brilhante.

"...sempre que formos precisar de que os dados sejam preservados entre uma chain e outro é necessário usar RunnablePassthrough.assign, pois ele é um sequencial de chains, conservando os dados de entrada naquela sequência específica."

Exatamente: o | (pipe) cria a sequência, e o .assign() em cada passo garante a conservação e o enriquecimento dos dados para a etapa seguinte.

Chegou a hora de juntar tudo. Abaixo está o código completo da nossa arquitetura final. Ele está autocontido e pronto para rodar. Logo em seguida, apresento a saída detalhada, explicando a origem de cada parte do resultado, como você pediu.

In [ ]:
import os
import datetime
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv, find_dotenv

# Carregar variáveis de ambiente (necessário para a chave da API)
# Certifique-se de que seu .env está configurado para o LangSmith
load_dotenv(find_dotenv())

# --- ATO I: DEFINIÇÃO DAS ESTRUTURAS DE DADOS (Nossos "Contratos") ---
class MovieInfoData(BaseModel):
    diretor: str = Field(..., description="Nome do diretor do filme")
    atores: List[str] = Field(..., description="Lista dos 3 atores principais do filme")
    ano: int = Field(..., description="Ano de lançamento do filme")

class Filmografia(BaseModel):
    filmes: List[str] = Field(description="Lista de filmes notáveis em que o ator participou")

# --- ATO II: CONSTRUÇÃO DAS "FERRAMENTAS" (Nossas Chains Independentes) ---

# Configuração do LLM que será usado por todas as ferramentas
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Ferramenta 1: Chain para buscar os dados iniciais do filme
prompt_movie_data = ChatPromptTemplate.from_template(
    "Você é um especialista em cinema. Forneça o diretor, o ano e a lista dos 3 atores principais do filme {movie}."
)
chain_busca_dados_filme = prompt_movie_data | llm.with_structured_output(MovieInfoData)

# Ferramenta 2: Chain para buscar o título em inglês
prompt_titulo_en = ChatPromptTemplate.from_template(
    "Você é um pesquisador especialista em cinema. Qual é o título original em inglês do filme '{movie}'? Responda apenas com o título, nada mais."
)
chain_busca_titulo_en = prompt_titulo_en | llm | StrOutputParser()

# Ferramenta 3: Chain para buscar a filmografia de um ator
ano_atual = datetime.datetime.now().year
prompt_filmografia = ChatPromptTemplate.from_template(
    """Você é um especialista em filmografia de atores.
    Liste filmes notáveis em que o ator {ator} participou, lançados estritamente depois do ano {ano_inicio} até o ano {ano_corrente}.
    Não inclua o filme do ano {ano_inicio}. Responda apenas com a lista de filmes."""
)
chain_busca_filmografia = prompt_filmografia | llm.with_structured_output(Filmografia)

# --- ATO III: MONTAGEM DA LINHA DE PRODUÇÃO (A Pipeline Completa) ---

# Etapa 1: Busca paralela dos dados iniciais e do título em inglês
etapa_1_busca_paralela = RunnableParallel({
    "dados_filme": chain_busca_dados_filme,
    "titulo_en": chain_busca_titulo_en,
})

# Etapa 2: Extração sequencial do ator e ano a partir dos dados da Etapa 1
etapa_2_extracao = RunnablePassthrough.assign(
    ator_principal=(lambda x: x["dados_filme"].atores[0]),
    ano_filme=(lambda x: x["dados_filme"].ano)
)

# Etapa 3: Busca sequencial da filmografia usando os dados extraídos na Etapa 2
adaptador_para_filmografia = lambda x: {
    "ator": x["ator_principal"],
    "ano_inicio": x["ano_filme"],
    "ano_corrente": ano_atual
}
etapa_3_filmografia = RunnablePassthrough.assign(
    filmografia_resultado=(adaptador_para_filmografia | chain_busca_filmografia)
)

# Montagem da Pipeline Completa (O Gerente Geral)
pipeline_completa = etapa_1_busca_paralela | etapa_2_extracao | etapa_3_filmografia


# --- ATO IV: EXECUÇÃO E ANÁLISE DO RESULTADO ---
input_data = {"movie": "Um Sonho de Liberdade"}
resultado_final = pipeline_completa.invoke(input_data)


# --- ANÁLISE DETALHADA DO RESULTADO FINAL ---
print("="*60)
print("INÍCIO DA ANÁLISE DA PIPELINE")
print("="*60)

print(f"\n➡️ 1. DADO DE ENTRADA (INPUT)")
print(f"   - {input_data}")

print("\n\n✅ 2. DICIONÁRIO FINAL (OUTPUT ENRIQUECIDO)")

# Iterando sobre o dicionário final para explicar a origem de cada chave
for chave, valor in resultado_final.items():
    if chave == "dados_filme":
        print(f"\n   - Chave '{chave}':")
        print(f"     - Origem: Gerado pela 'chain_busca_dados_filme' dentro da Etapa 1 (Paralela).")
        print(f"     - Valor: {valor}")
    elif chave == "titulo_en":
        print(f"\n   - Chave '{chave}':")
        print(f"     - Origem: Gerado pela 'chain_busca_titulo_en' dentro da Etapa 1 (Paralela).")
        print(f"     - Valor: '{valor}'")
    elif chave == "ator_principal":
        print(f"\n   - Chave '{chave}':")
        print(f"     - Origem: Extraído por uma 'lambda' do campo 'atores' na Etapa 2 (Sequencial).")
        print(f"     - Valor: '{valor}'")
    elif chave == "ano_filme":
        print(f"\n   - Chave '{chave}':")
        print(f"     - Origem: Extraído por uma 'lambda' do campo 'ano' na Etapa 2 (Sequencial).")
        print(f"     - Valor: {valor}")
    elif chave == "filmografia_resultado":
        print(f"\n   - Chave '{chave}':")
        print(f"     - Origem: Gerado pela 'chain_busca_filmografia' na Etapa 3 (Sequencial).")
        print(f"     - Valor: {valor.filmes}")

print("\n" + "="*60)
print("FIM DA ANÁLISE")
print("="*60)